In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import time

In [18]:
# Open a new file to save trace runs
col_names = ['Class','Alcohol', 'Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315 of diluted wines','Proline']
wine = pd.read_csv('wine.data', header = None)
wine.columns = col_names


# Normalize the array because large values would overwhem small values
wine_array = wine.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(wine_array)
df = pd.DataFrame(x_scaled)
wine = pd.DataFrame(x_scaled)
del df[0] # delete the first column ( Class )

k = 3 # for now we pick 3 clusters/centroids, later we'll check with the elbow method if this is the best solution

### Map Reduce

In [19]:
# we pick randomly our initial centroids
np.random.seed(2) # set a random seed so that we can reproduce the results
k_means = (df.sample(k, replace=False))    # store current means
k_means.index = pd.RangeIndex(len(k_means.index)) # re-assign index
k_means2 = pd.DataFrame()                    # store previous means
clusters = pd.DataFrame()                    # store distances
n_iter = 100 
n = 0

In [20]:
def mapper (k_means, dataframe):
    cluster_count = 0
    for idx, k_mean in k_means.iterrows():

        clusters[cluster_count] = (dataframe[k_means.columns] - np.array(k_mean)).pow(2).sum(1).pow(0.5)
        cluster_count += 1

    # update cluster
    dataframe['Cluster'] = clusters.idxmin(axis=1) # get's the minimum value of the cluster
    return dataframe

In [21]:
def reducer (k_means, k_means2, dataframe):

    if (k_means2.equals(k_means) == False):
        
        k_means2 = k_means
        # store previous cluster
        k_means = pd.DataFrame()
        k_means_frame = dataframe.groupby('Cluster').agg(np.mean)

        k_means[k_means_frame.columns] = k_means_frame[k_means_frame.columns]
    
        return k_means, '', k_means2

    else:
        return k_means, 'break', k_means2

In [22]:
# Timeit function

start_time = time.time()
while n < n_iter:
        # map function
        df1 = mapper (k_means, df)

        # reduce function      
        k_means, check, k_means2 = reducer (k_means, k_means2, df1)
        
        #dumbo.run(mapper, reducer, combiner=reducer)

        if check == 'break': # if centroids are equal to the previous ones we stop the process
            break
        else:
            n += 1
print("--- %s seconds ---" % (time.time() - start_time), "and we had {} iterations".format(n))

--- 0.20049667358398438 seconds --- and we had 5 iterations


In [93]:
df1.head(5)
pool.apply

,1,2,3,4,5,6,7,8,9,10,11,12,13,Cluster
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0
